Test Timer for sampling 

In [10]:

import os
import sys
import time
import serial
import math
import numpy as np
from tqdm import tqdm
from statistics import mean
import pandas as pd
from scipy.interpolate import interp1d
from local_occupancy import sensor_measurements, sensor_distances
from local_occupancy import thymio_coords, sensor_pos_from_center, sensor_angles
%matplotlib inline

sys.path.insert(0, os.path.join(os.getcwd(), 'src'))

from Thymio import Thymio

test_functions = False

th = Thymio.serial(port="COM8", refreshing_rate=0.1)

time.sleep(3) # To make sure the Thymio has had time to connect

variables = th.variable_description()

for var in variables : 
    print(var)

motors' speed:  [0, 0]
motors' speed:  [-4, 0]
motors' speed:  [0, 0]
{'name': '_id', 'offset': 0, 'size': 1}
{'name': 'event.source', 'offset': 1, 'size': 1}
{'name': 'event.args', 'offset': 2, 'size': 32}
{'name': '_fwversion', 'offset': 34, 'size': 2}
{'name': '_productId', 'offset': 36, 'size': 1}
{'name': 'buttons._raw', 'offset': 37, 'size': 5}
{'name': 'button.backward', 'offset': 42, 'size': 1}
{'name': 'button.left', 'offset': 43, 'size': 1}
{'name': 'button.center', 'offset': 44, 'size': 1}
{'name': 'button.forward', 'offset': 45, 'size': 1}
{'name': 'button.right', 'offset': 46, 'size': 1}
{'name': 'buttons._mean', 'offset': 47, 'size': 5}
{'name': 'buttons._noise', 'offset': 52, 'size': 5}
{'name': 'prox.horizontal', 'offset': 57, 'size': 7}
{'name': 'prox.comm.rx._payloads', 'offset': 64, 'size': 7}
{'name': 'prox.comm.rx._intensities', 'offset': 71, 'size': 7}
{'name': 'prox.comm.rx', 'offset': 78, 'size': 1}
{'name': 'prox.comm.tx', 'offset': 79, 'size': 1}
{'name': 'pro

In [11]:
from threading import Timer

class RepeatedTimer(object):
    def __init__(self, interval, function, *args, **kwargs):
        self._timer     = None
        self.interval   = interval
        self.function   = function
        self.args       = args
        self.kwargs     = kwargs
        self.is_running = False
        self.start()

    def _run(self):
        self.is_running = False
        self.start()
        self.function(*self.args, **self.kwargs)

    def start(self):
        if not self.is_running:
            self._timer = Timer(self.interval, self._run)
            self._timer.start()
            self.is_running = True

    def stop(self):
        self._timer.cancel()
        self.is_running = False


motors' speed:  [0, 0]
motors' speed:  [-5, -2]
motors' speed:  [-7, 0]
motors' speed:  [0, 0]


In [12]:
from time import sleep
Ts=1
value_proximity=0
value_acceleration=0
value_speed=0

motors' speed:  [-1, 0]
motors' speed:  [-3, 0]
motors' speed:  [-3, 0]
motors' speed:  [-3, 0]
motors' speed:  [0, 0]


In [18]:
def print_sensor_values():
    global value_proximity
    global value_acceleration
    global value_speed
    value_proximity=th['prox.horizontal']
    value_acceleration=th['acc']
    value_speed=[th['motor.left.speed'],th['motor.right.speed']]
    for i in range(2):
        if value_speed[i]>600:
            value_speed[i]=value_speed[i]-2**16
    for i in range(3):
        if value_acceleration[i]>600:
            value_acceleration[i]=value_acceleration[i]-2**16
    
    
    
    #print("proximity: ",value_proximity)
    print("accelaration: ",value_acceleration)
    #print("motors' speed: ",value_speed)


#print_sensor_values('prox.horizontal')

motors' speed:  [-4, 0]
motors' speed:  [-5, 0]


In [19]:
rt = RepeatedTimer(Ts, print_sensor_values)


#sensor_vals=print_sensor_values("prox.horizontal")
#print(sensor_vals)

motors' speed:  [0, 0]
motors' speed:  [-3, 0]


In [20]:
def move(l_speed=500, r_speed=500, verbose=False):
    """
    Sets the motor speeds of the Thymio 
    param l_speed: left motor speed
    param r_speed: right motor speed
    param verbose: whether to print status messages or not
    """
    # Printing the speeds if requested
    if verbose: print("\t\t Setting speed : ", l_speed, r_speed)
    
    # Changing negative values to the expected ones with the bitwise complement
    l_speed = l_speed if l_speed>=0 else 2**16+l_speed
    r_speed = r_speed if r_speed>=0 else 2**16+r_speed

    # Setting the motor speeds
    th.set_var("motor.left.target", l_speed)
    th.set_var("motor.right.target", r_speed)

accelaration:  [0, 0, 21]
motors' speed:  [0, 0]
motors' speed:  [0, 0]


In [21]:
move(-100,100,True)

		 Setting speed :  -100 100
accelaration:  [0, 0, 20]
motors' speed:  [-78, 85]
motors' speed:  [-81, 97]
accelaration:  [0, 0, 21]
motors' speed:  [-107, 98]
motors' speed:  [-101, 103]
accelaration:  [0, 0, 20]
motors' speed:  [-99, 99]
motors' speed:  [-101, 101]
accelaration:  [0, 0, 22]
motors' speed:  [-98, 100]
motors' speed:  [-100, 102]
accelaration:  [0, 0, 21]
motors' speed:  [-93, 102]
motors' speed:  [-98, 105]
accelaration:  [0, -3, 27]
motors' speed:  [-114, 98]
motors' speed:  [-113, 88]
accelaration:  [15, -5, 6]
motors' speed:  [-94, 102]
motors' speed:  [-97, 94]
accelaration:  [0, -3, 21]
motors' speed:  [-99, 103]
motors' speed:  [-102, 99]
accelaration:  [2, -3, 21]
motors' speed:  [-97, 102]
motors' speed:  [-110, 101]
accelaration:  [1, -3, 19]
motors' speed:  [-100, 102]
motors' speed:  [-101, 93]
accelaration:  [0, -3, 21]
motors' speed:  [-99, 102]
motors' speed:  [-104, 103]
accelaration:  [0, -3, 21]
motors' speed:  [-94, 100]
motors' speed:  [-94, 99]


In [22]:
move(0,0,True)

		 Setting speed :  0 0
accelaration:  [0, -3, 21]
motors' speed:  [-5, 0]
motors' speed:  [-2, 0]
accelaration:  [-4, -5, 12]
motors' speed:  [-4, 0]
motors' speed:  [0, 0]
accelaration:  [-1, 0, 21]
motors' speed:  [0, 0]
motors' speed:  [0, 0]
accelaration:  [9, 7, 24]
motors' speed:  [0, 0]
motors' speed:  [-8, 0]
accelaration:  [0, 4, 21]
motors' speed:  [-4, 0]
motors' speed:  [-8, 0]
accelaration:  [-6, -3, 14]
motors' speed:  [-3, 0]
motors' speed:  [-2, 0]
accelaration:  [13, 11, 27]
motors' speed:  [-7, 0]
motors' speed:  [-4, 0]
accelaration:  [4, 2, 9]
motors' speed:  [-3, 0]
motors' speed:  [0, 0]
accelaration:  [-2, -1, 24]
motors' speed:  [-3, 0]
motors' speed:  [-4, 0]
accelaration:  [18, 9, 14]
motors' speed:  [-3, 0]
motors' speed:  [-4, 0]
accelaration:  [-2, -7, -2]
motors' speed:  [-6, 0]
motors' speed:  [-9, 0]
accelaration:  [-5, -1, 20]
motors' speed:  [-5, 0]
motors' speed:  [-3, 0]
accelaration:  [0, 0, 11]
motors' speed:  [-5, 0]
motors' speed:  [-1, 0]
accel